## Step 1: Using SeqIO, read in and parse the file of apple primary transcripts (available on Canvas):
    - During your parsing, create a list of only transcripts of length 125 or less
    - Report the number of transcripts matching this criteria
    - Translate these sequences to protein, make sure to save them to their own list

In [1]:
from Bio import SeqIO

equal_or_less_125 = []  # a list of only transcripts of length 125 or less
fh = open("./Mdomestica_491_v1.1.cds_primaryTranscriptOnly.fa",'r')

for record in SeqIO.parse(fh,"fasta"):
    #print(record)
    if len(record) <= 125:
        equal_or_less_125.append(record)
    else:
        continue
        
print("The number of transcripts matching this criteria of length 'equal-to-or-less-than-125' is " + str(len(equal_or_less_125)))
      
translated = []
for p in equal_or_less_125:
    translated.append(p.translate())

The number of transcripts matching this criteria of length 'equal-to-or-less-than-125' is 62


#### The following cells are me feeling for what I could do

In [ ]:
#equal_or_less_125[0]  
translated[0]

In [ ]:
translated[0].seq

In [ ]:
type(translated[0].seq)

## Step 2: Align our small proteins using pairwise2
    - Align each protein to each other protein using the blosum62 scoring matrix, a gap opening penalty of -10, and a gap extension penalty of -0.5
    - If an alignment scores better than 40, save it.
    - Print the highest scoring pairwise alignment
    - Take care in how you set up your loops for the pairwise alignments. Each alignment only needs to be done once. If you've already tested the alignment of seq1 vs seq5, don't align seq5 vs seq1.
    

In [2]:
#for p in equal_or_less_125:
#    translated.append(p.translate())

In [3]:
#len(translated)
#translated[0:3]

In [4]:
from Bio import pairwise2
from Bio.SubsMat.MatrixInfo import blosum62

kept_alignments = []
highest_scoring_alignment = None
score_placeholder = None
placement = 0


for Seq_1 in translated:
    for Seq_2 in translated[placement:]:
        
        if Seq_1.seq != Seq_2.seq:
            align = pairwise2.align.localds(Seq_1.seq.rstrip('*'), Seq_2.seq.rstrip('*'), blosum62, -10, -0.5)
            a,b,c,d,e = align[0] # 'align' is a list of ONE object which is a FIVE variable tuple, the third variable is the SCORE. 
            
            if c >= 40:
                
                kept_alignments.append(align)
                
                
                if highest_scoring_alignment == None:
                    score_placeholder = c
                    highest_scoring_alignment = align
                    continue
                elif highest_scoring_alignment != None:
                    if c > score_placeholder:
                        score_placeholder = c
                        highest_scoring_alignment = align
                    else:
                        continue
                        
            else:
                continue
                
        else:
            continue
    
    placement += 1
    
    #placement = 1 
    #translated.pop(0) #This is to prevent redundant alignment pairing: i.e. 'Seq_1 to Seq_2' and 'Seq_2 to Seq_1'

In [5]:
len(kept_alignments)

2

In [6]:
print(highest_scoring_alignment)

[('MIWFCFVFLGNGVVRFSQLLGCHFHVGSTSTKLRVRICKI-', 'MIW-CFVELMFIVYGFACILLCIPTLGIQGTKFRSISNFLF', 46.0, 0, 34)]


In [7]:
kept_alignments[0][0]

('MVGVPQVFRRGELPNEVSCLL-----LIC---QSNESLFRSQIAKSSS',
 '---MPVVLR--ELDLESSCMVKFTVDLLCYELRSSWIVFRPQTRK---',
 40.5,
 3,
 45)

In [8]:
kept_alignments[1][0]

('MIWFCFVFLGNGVVRFSQLLGCHFHVGSTSTKLRVRICKI-',
 'MIW-CFVELMFIVYGFACILLCIPTLGIQGTKFRSISNFLF',
 46.0,
 0,
 34)

#### The following cells are me feeling for what I could do

In [ ]:
#from Bio import pairwise2
#from Bio.SubsMat.MatrixInfo import blosum62

In [ ]:
#translated[0].seq.rstrip('*')

In [ ]:
#seq_1 = translated[0].seq

In [ ]:
#test_align = pairwise2.align.localds(translated[0].seq.rstrip('*'), translated[1].seq.rstrip('*'), blosum62, -10, -0.5)

In [ ]:
#test_align

In [ ]:
#a,b,c,d,e = test_align[0]
#print(c)

In [ ]:
#print(pairwise2.format_alignment(*test_align[0]))

## Step 3: Running BLAST and reading results
     *Because we are doing a web BLAST, choose ONLY one sequence from our list of short proteins*.
    - Run BLAST with your sequence against the NR database
    - Parse the results. Report any HSPs with an E-value less than 0.05 and show the HSP alignments, including the name of the matching sequence. If no HSPs meet that criteria, report the highest scoring pair.
    - If for whatever reason the sequence you selected fails to return any results, try a new one 

In [ ]:
from Bio.Blast import NCBIWWW
results_handle = NCBIWWW.qblast('blastp','nr',kept_alignments[0][0])

In [ ]:
with open('BLAST_results.xml','w') as out_handle:
    out_handle.write(results_handle.read())
    
results_handle.close()

In [9]:
RESULTS_HANDLE = open('./BLAST_results.xml','r')

In [10]:
#RESULTS_HANDLE.close()

In [11]:
from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(RESULTS_HANDLE)

In [12]:
E_VALUE_THRESH = 0.05

for blast_record in blast_records:
    
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < E_VALUE_THRESH:
                print("****Alignment****")
                print("sequence:", alignment.title)
                print("length:", alignment.length)
                print("e value:",hsp.expect)
                print(hsp.query[0:75] + "...")
                print(hsp.match[0:75] + "...")
                print(hsp.sbjct[0:75] + "...")

****Alignment****
sequence: gb|RXH96343.1| hypothetical protein DVH24_008847 [Malus domestica]
length: 197
e value: 2.04733e-16
MPVVLRELDLESSCMVKFTVDLLCYELRSSWIVFRPQ...
MPVVLRELDLESSCMVKFTVDLLCYELRSSWIVFRPQ...
MPVVLRELDLESSCMVKFTVDLLCYELRSSWIVFRPQ...
****Alignment****
sequence: gb|RXI09171.1| hypothetical protein DVH24_023332 [Malus domestica]
length: 399
e value: 1.20114e-07
MVGVPQVFRRGELPNEVSCLLLICQ...
MVGVPQVFRRGELPNEVSCLLLICQ...
MVGVPQVFRRGELPNEVSCLLLICQ...
